### Deep Learning Model Evaluation

- Epoch: How many times the model will see the whole $X_{train}$ and $Y_{train}$. (How many iterations over the dataset)

- Almost always, we are interested to plot the training accuracy and testing (validation set) accuracy over epochs

- Also, we are interested to see the loss is decreasing over epochs. Trying to minimize our loss / cost function.

- How can we see these trends in Keras? How can we plot accuracy vs epoch for training and validation sets?

In [1]:
from tensorflow.keras.datasets import mnist
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.layers import LSTM
import tensorflow.keras

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train/np.max(x_train)
x_test = x_test/np.max(x_test)
y_train = tensorflow.keras.utils.to_categorical(y_train, 10)
y_test = tensorflow.keras.utils.to_categorical(y_test, 10)

# print(x_train[1])
x_train = x_train.reshape(x_train.shape[0], 28, 28)
x_test = x_test.reshape(x_test.shape[0], 28, 28)
print(x_train[0])
nb_units = 50

model = Sequential()
# model.add(LSTM(256, input_dim=1, input_length=5))
model.add(LSTM(nb_units, input_shape=(28, 28)))
model.add(Dense(units=10, activation='softmax'))
# 2.5 Compile the model.
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# 2.6 Print out model.summary
epochs = 3

# history = model.fit(x_train,
#                     y_train,
#                     epochs=epochs,
#                     batch_size=32,
#                     verbose=1,
#                     validation_split=0.2)

# scores = model.evaluate(x_test, y_test, verbose=2)
# print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

[[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.    

In [2]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 50)                15800     
_________________________________________________________________
dense (Dense)                (None, 10)                510       
Total params: 16,310
Trainable params: 16,310
Non-trainable params: 0
_________________________________________________________________


In [3]:
import matplotlib.pyplot as plt
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

NameError: name 'history' is not defined

In [4]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

NameError: name 'history' is not defined

In [5]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.callbacks import TensorBoard

tensor_board = TensorBoard(log_dir='./Graph/32')

# Set callback functions to early stop training and save the best model so far
callbacks = [EarlyStopping(monitor='val_loss', patience=2),
             ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True),
            tensor_board]

In [6]:
# Train neural network
history = model.fit(x_train, # Features
                      y_train, # Target vector
                      epochs=3, # Number of epochs
                      callbacks=[tensor_board], # Early stopping
                      verbose=0, # Print description after each epoch
                      batch_size=32, # Number of observations per batch
                      validation_data=(x_test, y_test)) # Data for evaluation

### Saving and loading a Keras model

- Given that deep learning models can take hours, days, or weeks to train, it is paramount to know how to save and load them from disk.

- Option 1: Save both the weights & the model architecture
- Option 2: Save or load the entire model

Weights can be saved in h5 format, and model architecture in JSON format.

In [7]:
from tensorflow.keras.models import model_from_json

# Option 1: Save Weights + Architecture
model.save_weights('model_weights.h5')
with open('model_architecture.json', 'w') as f:
    f.write(model.to_json())
    
# Option 1: Load Weights + Architecture
with open('model_architecture.json', 'r') as f:
    new_model_1 = model_from_json(f.read())
new_model_1.load_weights('model_weights.h5')

In [8]:
from tensorflow.keras.models import load_model

# Option 2: Save/load the entire model

# Creates a HDF5 file 'my_model.h5'
model.save('my_model.h5')

# Deletes the existing model
del model  

# Returns a compiled model identical to the previous one
model = load_model('my_model.h5')

In [10]:
from tensorflow.keras.utils import plot_model
import pydot
import graphviz

plot_model(model, to_file='model.png', show_shapes=True)

ModuleNotFoundError: No module named 'pydot'

What if we want to do audio classification? We can use transforms to turn the audio into an image using Fourier transforms, either the regular one or the STFT (also known as a spectogram). So image classification can also be applied to speech recognition!